In [1]:
!pip install -q transformers datasets accelerate peft torchvision
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.2 MB/s eta 0:00:00


In [2]:
# %% [markdown]
# ### 2. Setup Environment
# %%
import torch
from transformers import (
    Blip2Processor,
    Blip2ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from PIL import Image
import os

In [3]:
# Check GPU configuration
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")
print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f}GB")

GPU available: True
GPU name: Tesla T4
GPU memory: 14.75GB


In [4]:
# %% [markdown]
# ### 3. Load Model with Memory Optimizations
# %%
model_id = "Salesforce/blip2-opt-2.7b"
processor = Blip2Processor.from_pretrained(model_id)

# Load model with 8-bit quantization
model = Blip2ForConditionalGeneration.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto",
    # torch_dtype=torch.float16
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [6]:
# Freeze base model layers
#for param in model.vision_model.parameters():
#    param.requires_grad = False
#for param in model.qformer.parameters():
#    param.requires_grad = False
#for param in model.language_projection.parameters():
#    param.requires_grad = False

# Add LoRA adapters
lora_config = LoraConfig(
    r=64,  # Reduced from 16 for memory savings
    lora_alpha=32,
    lora_dropout=0.05,
    bias="all",
    #target_modules=["language_model.model.decoder.layers.*.self_attn.v_proj"]
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 22,629,760 || all params: 3,765,733,376 || trainable%: 0.6009


In [7]:
from torch.utils.data import Dataset, DataLoader
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["jpg"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        json_items = item["json"]
        encoding["text"] = json_items["prompt"]
        return encoding

def collate_fn(batch):
    # pad the input_ids and attention_mask
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch

In [8]:
from datasets import load_dataset

dataset = load_dataset("jackyhate/text-to-image-2M", split="train", streaming=True)

limited_dataset = []
for i, example in enumerate(dataset):
    if i < 100:
        limited_dataset.append(example)
    else:
        break

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

In [10]:
from datasets import Dataset as dataset_2

train_ds = limited_dataset[0:80]
eval_ds = limited_dataset[80:100]
train_ds_data = dataset_2.from_list(train_ds)
eval_ds_data = dataset_2.from_list(eval_ds)

train_ds_data = train_ds_data.remove_columns(["__key__", "__url__"])
eval_ds_data = eval_ds_data.remove_columns(["__key__", "__url__"])

In [11]:
train_dataset = ImageCaptioningDataset(train_ds_data, processor)
eval_dataset = ImageCaptioningDataset(eval_ds_data, processor)

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=1, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, collate_fn=collate_fn, batch_size=1)

In [13]:
import torch
from torch.utils.data import DataLoader
from bitsandbytes.optim import Adam8bit  # Must use 8-bit optimizer!

optimizer = Adam8bit(model.parameters(), lr=5e-4)  # Not regular Adam!
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 3. Training loop with FP16 cast
model.train()
for epoch in range(200):
    print(f"Epoch: {epoch}")
    for batch in train_dataloader:
        # Move batch to device WITH FP16 CASTING
        pixel_values = batch["pixel_values"].to(device, dtype=torch.float16)  # ← Must stay float16
        input_ids = batch["input_ids"].to(device)

        # Forward pass
        outputs = model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            labels=input_ids
        )

        # Backward pass
        loss = outputs.loss
        loss.backward()

        # Optimizer step
        optimizer.step()
        optimizer.zero_grad()

        print(f"Loss: {loss.item():.4f}")

Epoch: 0
Loss: 2.2734
Loss: 2.9121
Loss: 2.4707
Loss: 2.8613
Loss: 2.4336
Loss: 1.9072
Loss: 2.3613
Loss: 2.6504
Loss: 2.1777
Loss: 2.3535
Loss: 2.3203
Loss: 2.1191
Loss: 2.7578
Loss: 2.7871
Loss: 2.3398
Loss: 2.1445
Loss: 2.7559
Loss: 2.0723
Loss: 1.8652
Loss: 2.1641
Loss: 1.9141
Loss: 2.4375
Loss: 2.1777
Loss: 1.9854
Loss: 2.1406
Loss: 1.9609
Loss: 2.1953
Loss: 2.5527
Loss: 2.3789
Loss: 2.1992
Loss: 2.1680
Loss: 2.0195
Loss: 2.1758
Loss: 2.4277
Loss: 1.9512
Loss: 2.0645
Loss: 2.1992
Loss: 1.8242
Loss: 1.9072
Loss: 1.9736
Loss: 2.1328
Loss: 2.3828
Loss: 1.9609
Loss: 1.9482
Loss: 2.6133
Loss: 2.7676
Loss: 3.0703
Loss: 1.8174
Loss: 2.0938
Loss: 2.2734
Loss: 2.4453
Loss: 1.6855
Loss: 2.2148
Loss: 2.6172
Loss: 2.2422
Loss: 1.8848
Loss: 1.7529
Loss: 1.8779
Loss: 1.8145
Loss: 2.6426
Loss: 2.2051
Loss: 2.2988
Loss: 2.0488
Loss: 2.6816
Loss: 2.5938
Loss: 2.1836
Loss: 1.8057
Loss: 1.9600
Loss: 1.6973
Loss: 1.7637
Loss: 2.4160
Loss: 2.4766
Loss: 2.3145
Loss: 1.7100
Loss: 2.1055
Loss: 2.8691
Los

KeyboardInterrupt: 

In [17]:
model.eval()  # ← Crucial!
val_loss = 0
with torch.no_grad():
    for batch in eval_dataloader:  # You'll need to create eval_loader
        pixel_values = batch["pixel_values"].to(device, torch.float16)
        input_ids = batch["input_ids"].to(device)


        outputs = model(pixel_values=pixel_values,
                        input_ids=input_ids,
                        labels=input_ids)
        val_loss += outputs.loss.item()

val_loss /= len(eval_dataloader)
print(f"Validation Loss: {val_loss:.4f}")  # Compare with training loss

Validation Loss: 2.7340


In [18]:
from huggingface_hub import notebook_login
notebook_login()  # Opens a widget to input token


In [19]:
# Push model with your username
model.push_to_hub("Baran657/blip_2_multimedia")

# Push processor (optional but recommended)
processor.push_to_hub("Baran657/blip_2_multimedia")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Baran657/blip_2_multimedia/commit/71e2a8480fc7cefd1d446694cd3c8a085722997a', commit_message='Upload processor', commit_description='', oid='71e2a8480fc7cefd1d446694cd3c8a085722997a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Baran657/blip_2_multimedia', endpoint='https://huggingface.co', repo_type='model', repo_id='Baran657/blip_2_multimedia'), pr_revision=None, pr_num=None)

In [41]:
model.eval()

image = limited_dataset[0]["jpg"]
image = image.convert("RGB")

question = "What can you see?"
inputs = processor(image, question, return_tensors="pt").to("cuda")

out = model.generate(
    **inputs,
    max_new_tokens=80,  # Extend from default 30 to ~100 words
    num_beams=5,         # Better than greedy search
    repetition_penalty=5.0,  # Reduce redundancy
    temperature=0.4,     # Balance creativity/factuality
    early_stopping=True
)
print(processor.decode(out[0], skip_special_tokens=True).strip())

#out = model.generate(pixel_values=pixel_values, input_ids=input_ids, return_tensors="pt")

#print(processor.decode(out[0], skip_special_tokens=True).strip())

What can you see? camon storage bag with a mesh panel on top, and a handle attached to it. The bag has two handles and a zipper closure. The bag is placed against a white background, and the image is well-lit with a softbox light source. The text 'CAMON STORAGE BAG' is displayed at the top, along with the tagline 'Premium Material, Practical Design
